# Introduction and Context
![partially folded paper with geometry on it](folded_paper.jpeg)

Physics simulation is powerful but has a lot of moving parts. Data science tooling - and notebooks in particular -  are a rich way to orchestrate computational workflows, explore complex data spaces, and illuminate the resultant narratives. Our goal here is to provide a working example of one such workflow.

In particular,  A live Jupyter notebook is a wonderful primary interface for a simple workflow in Computational Fluid Dynamics (CFD). Beyond access to third party tools for problem specific computations and tasks it provides a central workbench to relate and process data in a familiar, consistent way using standard tools and approaches.

To demonstate a simple version of this claim, we will create and explore a simple parametric design space where a few numbers will characterize the geometric design and physical context of a paper airplane. 
Using simulation data we will explore some of these parameters to understand how the way we fold and throw the airplane may affect its flight performance.

With the ability to view ensembles of physical simulations as classic dataframes, we will do some of the things one normally does with dataframes. We will build models to interpolate and predict, we will use those models to optimize and analyse, and we will explore how to construct the dataframe for a given problem in the most efficient and efficaceous way. 

We can naturally partition this work into three sections:

* A Workflow for a Single Simulation
 - Geometry
 - Meshing
 - Simulation
 - analysis
* Dataframe centered Simulation Ensembles
 - Dataframes
 - Computation
 - Persistence
* CFD Ensemble Centered Applications
 - Surrogate Models and Estimation
 - AeroDBs (precomputed ensembles)
 - Optimization
 
The "Colophon" section describes the specific tools we use and how they are integrated so we treat them as abtracted "black boxes" in much of the remainder, and the "Open Questions and Future Work" sectio does what it says on the tin. 

## Audience

This article is likely most useful for two types of reader:  
* comfortable with data science tools with a basic knowledge or interest in physics simulation
* physics simulation practitioners wanting to use common datascience tools in their work. 

We will treat the actual simulation as something of a "black box": to us a given simulation run is just one mapping from the input space to the output space among many. Making this simulation achieve an optimal balance between cost and accuracy (of various sorts) is of course a deep area of expertise, involves problem specific understanding, and is beyond our scope here.

## Problem Space

This is intended for ensembles of simulations which may be naturally indexed by some reasonable number of parameters. These parameters will be columns in our data frame, along with any other configuration and physical data needed to run and analyze a single simulation. While we will focus on "interesting" parameters and a structured design space, the data frame can certainly just comprise an unstructured set of simulations but the output space should be consistent and comparable.


We will start with non-transient problems, but this is only for initial simplicity. We can easily add time indexing to this workflow.

## Solution Space


The full solution data should be available, but our data frame will only contain specific extracted outputs and metadata.

Calculated outputs and objective functions will be available for any completed simulation and will be added to the columns of our dataframe.  Note that this includes not only physical metrics (like lift and drag) but computational ones (like run time or stability). References to the full raw solution allows us to extend and recalculate these outputs or create new ones.


# A CFD Workflow for a Single Simulation

We begin by setting up the tools and models to run a single standalone simulation and analyze its results. We will use the following python and other executables. See the `README` or the individual `lightningcatcher_*` files for installation notes.

In [2]:
# Get notebook dir on path
import pathlib, sys
sys.path.append(pathlib.Path().absolute())

# python tooling
import subprocess, sys,os, shutil, math, types
import pandas
import numpy as np
from itertools import chain

# datascience tooling:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from IPython.display import Image, HTML
from mpl_toolkits.mplot3d import Axes3D

# CFD tooling:
import lightningcatcher_gmsh as geometry
import lightningcatcher_su2 as cfd
import lightningcatcher_paraview as vis

## Geometry

For our example we will model a standard paper airplane as descibed [in Wikipedia](https://en.wikipedia.org/wiki/Paper_plane).

This simple design starts with folding a paper length-wise. Then we make three repeated folds of the current front wing corner in to the center line. The last of these creates the wing surface, and can be varied to provide an interesting geometric parameter for our design space. That is, what we have annotated as $\phi$ on this  diagram:

![Wikipedia-sourced Plane Param](plane_param_diagram.jpg)

This $\phi$ will be a primary input parameter alongside the angle of attack: the "tilt" with which the airplane moves through the air.

Our output parameters will represent the aerodynamic performance of a given design in terms of Lift and Drag or their ratio. We can compare our simulation output against analytical and ground truth results using [existing work](http://www.lactea.ufpr.br/wp-content/uploads/2018/08/On_the_Aerodynamics_of_Paper_Airplanes.pdf)

There are many other parameters we can explore - you will see some below where define all the model variables - but these two provide an interesting but managable design space to explore.

Our design space includes a lot of things we likely won't vary: the size of the paper, the final "camber" angle of the wings, etc. We encode them all in a dict, with some algorithm and visualization parameters too for convenience. You will not be misled if you think of this as a row in a dataframe, or an input point in a sample space, but we won't do so here just yet. For now, we'll use a single config for geometric, algorithmic, and visualization parameters to take a single example conf from geometry through analysis.

In [3]:
no_display=False
config = {
    "meshsize_large": 5.0,           # uniform meshing for now
    "meshsize_small": 0.8,  
    "case_index": 0,  
    "base_dir": "/Users/scot/Projects/Personal/lightningcatcher/notebooks/",
    "sim_path": "/Users/scot/Projects/SU2/bin/SU2_CFD",
    "mesh_resolution": 0.2,
    "mesh_sizemin_scale": 5.0,
    "mesh_sizemax_scale": 25.0,
    "mesh_distmin_scale": 10.0,
    "mesh_distmax_scale": 50.0,
    "group_boundary_size": 6.0,
    "bounding_box_size": 15.0,    
    "full_fold_radians" : geometry.pi,     # usually pi, unless we fold loosely
    "wing_tilt_radians" : geometry.pi/2.0,  # wing angle with fuselage
    "camZoom":5.0,
    "camOffsetX":0.0,
    "camOffsetY":0.0,
    "case_name": f"case",      # overall project name (directory)
    "paper_size_x" : 11.0,                 # sheet of paper height
    "paper_size_z": 4.25,                  # paper width
    "paper_thickness":0.1, 
    "body_tilt_radians" : geometry.pi/8.0,  # main fuselage fold angle
    "camX":30.0,                     # Camera positioning
    "camY":60.0,
    "camZ":0.0,
    "camOffsetZ":0.0,  
    'project_name': "z",
    "mach_number": 0.01,
    "fuselage_radians" : 0.6, #geometry.pi/8.0,   # wing "slope" fold angle
    "angle_of_attack" : 12.0,        # orientation to wind/movement
}

To construct our model for a given parameter set we need only a "sheet" to start, a "fold" operation, and a "mirror" operation to save some time at the end. These should all be elementary operations in any geometry package, with a few cross products from `numpy`. We'll keep the impementation separate to make it easy to substitute different packages for the CAD and Meshing.

In [4]:
# We'll make this a function - for easy variable access to config values, but also for later work
def make_airplane_model(**config):
    
    # initialize the project, set camera view, and create the sheet of paper:
    c = types.SimpleNamespace(**config)
    geometry.initialize() #case_name)
    geometry.set_camera(c.camX, c.camY, 
                        c.camZ, c.camZoom, c.camZoom, c.camZoom,
                        c.camOffsetX, c.camOffsetY, c.camOffsetZ)
    
    # define points & angles from the above diagram, starting with corners of the paper
    p1,p2,p3,p4 = ( [0.0, 0.0, 0.0],
                    [0.0, 0.0, c.paper_size_z],
                    [c.paper_size_x, 0.0,  c.paper_size_z],
                    [c.paper_size_x, 0.0, 0.0])
    fold_val = c.paper_size_x*math.tan(geometry.pi/8.0)
    fixed_intercept = [c.paper_size_x, 0.0, fold_val] # c.paper_size_x*math.sin(gmsh.pi/8.0), 0.0]
    intercept = [c.paper_size_x, 0.0, c.paper_size_z*math.sin(c.fuselage_radians)]
    origin = [0.0, 0.0, 0.0]
    first_fold_pojnt = [c.paper_size_z, 0.0,c.paper_size_z]
    sheet=geometry.make_sheet(p1,p2,p3,p4, c.paper_thickness)
    
    # fold the sheet into the airplane:
    # two folds from the nose, then the final fold forms the wing
    fold1 = geometry.fold(sheet, origin, first_fold_pojnt, p2,c.full_fold_radians)
    fold2 = geometry.fold(fold1, origin,fixed_intercept, p2, c.full_fold_radians)  
    half = geometry.fold(fold2, origin,intercept,p2, c.wing_tilt_radians)
    
    # tilt a little for the "v" of the fusesage, then mirror.
    rhalf = geometry.fold(half, origin,p4, p2, c.body_tilt_radians)
    lhalf = geometry.copy(rhalf)
    geometry.mirror(lhalf, 0, 1,0, 0)
    
    # merge all the parts of the shape, remove internal surfaces, return model:
    geometry.synchronize()
    thing, lhb = geometry.fuse(rhalf, lhalf)
    config['geometry_index'] = thing
    return config

def merge_dicts(d1,d2):
    return dict(chain(d1.items(), d2.items()))

We can take a look at our model before proceeding, just using `gmsh` itself. This is a nice way to dial in our model before proceeding, since we can change any of the parameters easily when we call it. It's very powerful to explore changes to the mesh interactively then set the resulting parameters in our config above.

Let's try it out!

```
# camera_params=dict(camX=-5.0,camY=30.0,camZ=10.0,camZoom=2.0, camOffsetX=-0.0)
examples = []
example_geometries = [geometry.pi/x for x in [22.0, 8.0, 4.0] ]
for i in range(3):
    examples.append(cfd.single_simulation(config, make_airplane_model, do_meshing=False, run_simulation=False, 
                                         **dict(camX=-60.0,camY=0.0,camZ=-30.0,camZoom=1.5, camOffsetX=-0.0, meshsize_large=3.0,
                                               fuselage_radians=example_geometries[i], define_surface_groups=False,
                                               case_name=f"{config['project_name']}-{i:0>3}")))
                                               
```

In [ ]:
# merge_dicts(dict(a=3, b=4), dict(c=10, a=1))

# camera_params=dict(camX=-5.0,camY=30.0,camZ=10.0,camZoom=2.0, camOffsetX=-0.0)
examples = []
example_geometries = [geometry.pi/x for x in [22.0, 8.0, 4.0] ]
for i in range(3):
    c = merge_dicts(
        config, 
        dict(camX=-60.0,camY=0.0,camZ=-30.0,camZoom=1.5, camOffsetX=-0.0, meshsize_large=3.0,
                                               fuselage_radians=example_geometries[i], define_surface_groups=False,
                                               case_name=f"{config['project_name']}-{i:0>3}",
            #imagepath=f"{config['project_name']}/{config['project_name']}-{i:0>3}.jpg"
            ))
    c = make_airplane_model(**c)
    geometry.save_mesh(c,None, f"{config['project_name']}/{config['project_name']}-{i:0>3}.jpg")
    # geometry.finalize()
    examples.append(c)

In [ ]:
display(HTML(
    vis.html_imgtable(
        [f"Fuselage Angle: {e['fuselage_radians']:.2f} rad" for e in examples],
        [e['imagepath'] for e in examples])))

![Three design sample points](three_geo.png)

That looks good enough for now. Note we're leaving the paper unrealistically thick here. Further, this diagram makes clear that other geometric parameters - the angle of the fuselage "v" itself, for example - likely have significant aerodynamic impact.

For now though, let's sprint to trying to get some aero modelling done!

##  Meshing

We do only simplistic of meshing here.  This keeps the geometry very simple and leaves us with a great example of using this approach to improve our models in a later article. Since we're not tuning anything, we just need to tell it to do the meshing by setting `do_meshing=True`:

In [ ]:
e = examples[1]
e['geometry_index'] = make_airplane_model(**e)
c2 = geometry.add_mesh(e, e['geometry_index']) #, run_simulation=False, interactive=False,
                       #**dict(camX=-52,camY=38.8,camZ=0.0,camZoom=2.5, camOffsetX=-0.0))

print(e['imagepath'])

In [ ]:
display(Image(filename=c2['imagepath']) )

![Meshed geometry view](abc3/abc3-001.jpg)
abc1/case.jpg)

##  Simple Simulation - 3D flow in air at low velocity
We're ready to run a single simulation of this simple model. Beyond the raw solution flow data, we'll extract a few global output variables of interest.

In [ ]:
c3 = cfd.run_simulation(e, do_extract=['eff_coeff','lift_coeff', 'drag_coeff', 'momentum_z', 'momentum_x'])


## Analysis of a single simulation run

We ran our single simulation, but don't have much to look at yet. Let's look into how we can take a look at our results - numerically and visually.

#### Extract per-surface and global objective function values

Note that we update the "config" for each simulation with resultant output variables, so we have some output global data already available to us there. We'll look at the Aerodynamic Efficiency as a way to quantify "how good" a given design is - simply the ratio of Lift to Drag.

$$ \mathit{Aero\ efficiency} = \frac{\mathit{Lift}}{\mathit{Drag}} = \frac{\mathit{Lift\ Coefficient}}{\mathit{Drag\ Coefficient}}$$

In [ ]:
c3['lift_coeff']/c3['drag_coeff'], c3['eff_coeff']

#### Visualize single flow in `matplotlib`

Using standard notebook 3D plotting can get us started visualizing the results of our simulation. Let's make a 3D scatter of our airplane model with the estimated force vector at each point. In `base.cfg` we set the output path for a surface flow file. As a first step toward building our own visualizations using `matplotlib` and standard DS tools, let's open up the surface flow CSV file we generate for each sim and extract global physics data.

In [ ]:
%matplotlib notebook
sdf=pandas.read_csv( os.path.join(config['project_name'], 'case-surface_flow.csv'))
fig2 = plt.figure()
ax21 = fig2.add_subplot(projection='3d', azim=20)
ax21.quiver(sdf['x'], sdf['y'],sdf['z'], sdf['Momentum_x'], sdf['Momentum_y'], sdf['Momentum_z'])
#fig2.show()
#plt.savefig("abc1/quiver.jpg")

! [Force on airplane surfaces](quiver.jpg)

#### Visualize flow solution using external vis package
It's great to know how to build simple data representations, but for more involved analysis we'll use a purpose built tool to look at and analyze our results. Let's generate a single vis frame for one of our sample space simulations, but note that we can interact with the visualization software interactively - as with the Geometry and Meshing, this can be invaluable in trying new things and exploring.

In [ ]:
c3['viz_path'] = vis.visualize_case(c3['project_name'], c3)

#display(Image(filename=c3['viz_path'] ))
print(c3['viz_path'])

![Flow visualization](case-para.png)

#  Dataframes for CFD Ensembles

Often we're concerned with a set of related simulation runs. How should we consider this element and set? A Data Science person might like to consider a single simulation as a row in a dataframe; a CFD practitioner might think of it as an element of a DOE; a designer may just have a bunch of experimenal runs to compare and relate.

It's nice sometimes to think of CFD simulation runs as simply another mapping from an input space to an output space. We may have multiple ways to construct this mapping - for instance: different algorithmic parameters and entire algorithms, precise vs fast CFD and even non-CFD estimates, even experimental data. So rows in the input space will not have unique images in the output space unless we include the mapping used as an input. It can be useful to seperate physical inputs (air temperature), design inputs (fuselage angle), model inputs (mesh size), and algorithmic ones (CFD, linear solver), but here we want to keep things as simple as possible so we'll keep all the inputs "flat" for now.

Each row of our dataframe will contain everything needed to run and analyze a single simulation - a sample from our input space. There may be template files and mesh files which are not part of the dataframe, generally platform specific, but everything needed to generate and use these files will be columns in our dataframe.



We want a dataframe with the input part of every row populated with interesting values. Then for a given mapping we can generate the outputs. We begin by specifying all the static parameters and then varying a subset - the DOE parameters, or a sampling method in the input space, or a set of values of interest. In this case, we will vary the `fuselage_rad` alone - then with `angle of attack` in a simple grid:

We'll uses a  function to add columns to the dataframe for our project, which can involve any of:
* run CFD simulation
* extract values from previously run simulation
* apply estimations (surrogate models) to generate values
* generate sumary and visualization data

So, the return value will be available as a dataframe row.

## Build the dataframe
### First create the rows from sample points:

### Just a test of two related sims

In [5]:
def save_all(c):
    return geometry.save_mesh(c,c['meshpath'], f"{c['project_name']}/{c['project_name']}-001.jpg")

In [9]:
e = config.copy() #examples[1]
e = make_airplane_model(**e)
e = geometry.add_mesh(e) 
#geometry.save_mesh(e,e['meshpath'], f"{e['project_name']}/{e['project_name']}-000.jpg")
e = save_all(e)
            
e = cfd.run_simulation(e, do_extract=['eff_coeff','lift_coeff', 'drag_coeff', 'momentum_z', 'momentum_x'])

e2 = config.copy() #examples[1]
e2['angle_of_attack'] = 5.0
e2["case_name"] = f"{e2['project_name']}-001"
e2 = make_airplane_model(**e2)
e2 = geometry.add_mesh(e2) 
save_all(e2)

e2 = cfd.run_simulation(e2, stdout_to_log=False,do_extract=['eff_coeff','lift_coeff', 'drag_coeff', 'momentum_z', 'momentum_x'])

Volume: [(3, 3)]
Config: {'meshsize_large': 5.0, 'meshsize_small': 0.8, 'case_index': 0, 'base_dir': '/Users/scot/Projects/Personal/lightningcatcher/notebooks/', 'sim_path': '/Users/scot/Projects/SU2/bin/SU2_CFD', 'mesh_resolution': 0.2, 'mesh_sizemin_scale': 5.0, 'mesh_sizemax_scale': 25.0, 'mesh_distmin_scale': 10.0, 'mesh_distmax_scale': 50.0, 'group_boundary_size': 6.0, 'bounding_box_size': 15.0, 'full_fold_radians': 3.141592653589793, 'wing_tilt_radians': 1.5707963267948966, 'camZoom': 5.0, 'camOffsetX': 0.0, 'camOffsetY': 0.0, 'case_name': 'case', 'paper_size_x': 11.0, 'paper_size_z': 4.25, 'paper_thickness': 0.1, 'body_tilt_radians': 0.39269908169872414, 'camX': 30.0, 'camY': 60.0, 'camZ': 0.0, 'camOffsetZ': 0.0, 'project_name': 'z', 'mach_number': 0.01, 'fuselage_radians': 0.6, 'angle_of_attack': 12.0, 'geometry_index': [(3, 3)]}
Volume: [(3, 1)]
Config: {'meshsize_large': 5.0, 'meshsize_small': 0.8, 'case_index': 0, 'base_dir': '/Users/scot/Projects/Personal/lightningcatcher/n

+-----------------------------------------------------------------------------+
|  Inner_Iter|   Time(sec)|    rms[Rho]|   rms[RhoE]|          CL|          CD|
+-----------------------------------------------------------------------------+
|           0|  2.2487e-02|   -1.747171|    2.650778|  -13.763619|    3.046278|
|           1|  2.2469e-02|   -2.127609|    2.273532|   -5.169636|    1.521292|
|           2|  2.2400e-02|   -2.148811|    2.249060|   -0.379328|    0.425168|
|           3|  2.2745e-02|   -2.382682|    2.014943|    0.486324|    0.289197|
|           4|  2.2716e-02|   -2.579633|    1.818103|    0.222564|    0.321979|
|           5|  2.2580e-02|   -2.742104|    1.657063|    0.065205|    0.306451|
|           6|  2.2478e-02|   -2.866965|    1.533595|   -0.018488|    0.294692|
|           7|  2.2448e-02|   -2.983756|    1.417592|   -0.066293|    0.291266|
|           8|  2.2407e-02|   -3.090624|    1.311267|   -0.091624|    0.287481|
|           9|  2.2496e-02|   -3.188250|

|         190|  2.2571e-02|   -3.218981|    1.178982|   -0.390101|    0.242073|
|         191|  2.2572e-02|   -3.214268|    1.183695|   -0.390361|    0.241992|
|         192|  2.2573e-02|   -3.209555|    1.188409|   -0.390624|    0.241910|
|         193|  2.2574e-02|   -3.204840|    1.193123|   -0.390890|    0.241828|
|         194|  2.2574e-02|   -3.200126|    1.197837|   -0.391160|    0.241745|
|         195|  2.2576e-02|   -3.195410|    1.202552|   -0.391432|    0.241661|
|         196|  2.2580e-02|   -3.190694|    1.207268|   -0.391708|    0.241576|
|         197|  2.2580e-02|   -3.185977|    1.211985|   -0.391988|    0.241490|
|         198|  2.2579e-02|   -3.181260|    1.216702|   -0.392270|    0.241403|
|         199|  2.2581e-02|   -3.176542|    1.221420|   -0.392556|    0.241316|
|         200|  2.2581e-02|   -3.171823|    1.226138|   -0.392846|    0.241227|
+-----------------------------------------------------------------------+
|        File Writing Summary       |         

|         372|  2.2783e-02|   -3.878484|    0.519451|   -0.370755|    0.247722|
|         373|  2.2783e-02|   -3.873752|    0.524183|   -0.370824|    0.247704|
|         374|  2.2783e-02|   -3.869025|    0.528912|   -0.370894|    0.247685|
|         375|  2.2784e-02|   -3.864300|    0.533637|   -0.370963|    0.247667|
|         376|  2.2784e-02|   -3.859578|    0.538360|   -0.371034|    0.247649|
|         377|  2.2784e-02|   -3.854857|    0.543081|   -0.371104|    0.247630|
|         378|  2.2783e-02|   -3.850139|    0.547801|   -0.371175|    0.247611|
|         379|  2.2782e-02|   -3.845421|    0.552519|   -0.371246|    0.247592|
|         380|  2.2782e-02|   -3.840705|    0.557236|   -0.371318|    0.247573|
|         381|  2.2780e-02|   -3.835989|    0.561952|   -0.371390|    0.247554|
|         382|  2.2779e-02|   -3.831275|    0.566668|   -0.371463|    0.247534|
|         383|  2.2778e-02|   -3.826560|    0.571383|   -0.371536|    0.247515|
|         384|  2.2776e-02|   -3.821846|

|         554|  2.2692e-02|   -4.547553|   -0.149637|   -0.366697|    0.248972|
|         555|  2.2691e-02|   -4.541783|   -0.143866|   -0.366712|    0.248966|
|         556|  2.2689e-02|   -4.536148|   -0.138231|   -0.366727|    0.248961|
|         557|  2.2687e-02|   -4.530630|   -0.132713|   -0.366742|    0.248956|
|         558|  2.2685e-02|   -4.525216|   -0.127298|   -0.366758|    0.248950|
|         559|  2.2683e-02|   -4.519891|   -0.121973|   -0.366774|    0.248945|
|         560|  2.2682e-02|   -4.514646|   -0.116727|   -0.366790|    0.248940|
|         561|  2.2680e-02|   -4.509469|   -0.111550|   -0.366807|    0.248935|
|         562|  2.2678e-02|   -4.504353|   -0.106433|   -0.366824|    0.248930|
|         563|  2.2676e-02|   -4.499289|   -0.101369|   -0.366841|    0.248925|
|         564|  2.2676e-02|   -4.494272|   -0.096351|   -0.366858|    0.248920|
|         565|  2.2676e-02|   -4.489296|   -0.091374|   -0.366876|    0.248915|
|         566|  2.2677e-02|   -4.484355|

...

|        1476|  2.2194e-02|   -7.088812|   -2.690895|   -0.365693|    0.249295|
|        1477|  2.2194e-02|   -7.076002|   -2.678084|   -0.365693|    0.249295|
|        1478|  2.2194e-02|   -7.064399|   -2.666481|   -0.365693|    0.249296|
|        1479|  2.2194e-02|   -7.053800|   -2.655882|   -0.365693|    0.249296|
|        1480|  2.2194e-02|   -7.044041|   -2.646123|   -0.365693|    0.249296|
|        1481|  2.2195e-02|   -7.034989|   -2.637071|   -0.365693|    0.249296|
|        1482|  2.2195e-02|   -7.026535|   -2.628617|   -0.365693|    0.249296|
|        1483|  2.2195e-02|   -7.018588|   -2.620671|   -0.365693|    0.249296|
|        1484|  2.2194e-02|   -7.011075|   -2.613158|   -0.365693|    0.249296|
|        1485|  2.2194e-02|   -7.003934|   -2.606017|   -0.365693|    0.249296|
|        1486|  2.2194e-02|   -6.997111|   -2.599194|   -0.365693|    0.249296|
|        1487|  2.2194e-02|   -6.990563|   -2.592646|   -0.365693|    0.249296|
|        1488|  2.2194e-02|   -6.984252|

|        1659|  2.2190e-02|   -6.927599|   -2.529646|   -0.365701|    0.249293|
|        1660|  2.2190e-02|   -6.922892|   -2.524939|   -0.365701|    0.249293|
|        1661|  2.2189e-02|   -6.918185|   -2.520231|   -0.365701|    0.249293|
|        1662|  2.2189e-02|   -6.913478|   -2.515524|   -0.365701|    0.249293|
|        1663|  2.2188e-02|   -6.908770|   -2.510817|   -0.365701|    0.249293|
|        1664|  2.2188e-02|   -6.904064|   -2.506110|   -0.365701|    0.249293|
|        1665|  2.2187e-02|   -6.899357|   -2.501403|   -0.365706|    0.249293|
|        1666|  2.2187e-02|   -7.539867|   -3.143602|   -0.365700|    0.249294|
|        1667|  2.2186e-02|   -7.603387|   -3.207785|   -0.365697|    0.249294|
|        1668|  2.2186e-02|   -7.723269|   -3.328210|   -0.365696|    0.249295|
|        1669|  2.2185e-02|   -7.877323|   -3.482304|   -0.365695|    0.249295|
|        1670|  2.2185e-02|   -8.035705|   -3.640470|   -0.365695|    0.249295|
|        1671|  2.2185e-02|   -8.140510|

|        1841|  2.2211e-02|   -7.588831|   -3.190887|   -0.365697|    0.249294|
|        1842|  2.2210e-02|   -7.584124|   -3.186179|   -0.365697|    0.249294|
|        1843|  2.2210e-02|   -7.579416|   -3.181471|   -0.365697|    0.249294|
|        1844|  2.2209e-02|   -7.574709|   -3.176763|   -0.365697|    0.249294|
|        1845|  2.2209e-02|   -7.570001|   -3.172055|   -0.365697|    0.249294|
|        1846|  2.2209e-02|   -7.565294|   -3.167347|   -0.365697|    0.249294|
|        1847|  2.2208e-02|   -7.560586|   -3.162639|   -0.365697|    0.249294|
|        1848|  2.2208e-02|   -7.555879|   -3.157931|   -0.365697|    0.249294|
|        1849|  2.2208e-02|   -7.551171|   -3.153223|   -0.365697|    0.249294|
|        1850|  2.2208e-02|   -7.546464|   -3.148515|   -0.365697|    0.249294|
|        1851|  2.2208e-02|   -7.541756|   -3.143807|   -0.365697|    0.249294|
|        1852|  2.2208e-02|   -7.537049|   -3.139099|   -0.365697|    0.249294|
|        1853|  2.2209e-02|   -7.532341|

|        2023|  2.2229e-02|   -8.251188|   -3.853265|   -0.365696|    0.249295|
|        2024|  2.2228e-02|   -8.246307|   -3.848383|   -0.365696|    0.249295|
|        2025|  2.2228e-02|   -8.241448|   -3.843523|   -0.365696|    0.249295|
|        2026|  2.2228e-02|   -8.236611|   -3.838686|   -0.365696|    0.249295|
|        2027|  2.2228e-02|   -8.231792|   -3.833866|   -0.365696|    0.249295|
|        2028|  2.2228e-02|   -8.226990|   -3.829063|   -0.365696|    0.249295|
|        2029|  2.2228e-02|   -8.222201|   -3.824273|   -0.365696|    0.249295|
|        2030|  2.2228e-02|   -8.217423|   -3.819494|   -0.365696|    0.249295|
|        2031|  2.2228e-02|   -8.212658|   -3.814728|   -0.365696|    0.249295|
|        2032|  2.2228e-02|   -8.207901|   -3.809971|   -0.365696|    0.249295|
|        2033|  2.2228e-02|   -8.203153|   -3.805221|   -0.365696|    0.249295|
|        2034|  2.2229e-02|   -8.198410|   -3.800478|   -0.365696|    0.249295|
|        2035|  2.2229e-02|   -8.193674|

|        2205|  2.2257e-02|   -8.961907|   -4.563989|   -0.365696|    0.249295|
|        2206|  2.2257e-02|   -8.950305|   -4.552387|   -0.365696|    0.249295|
|        2207|  2.2257e-02|   -8.939705|   -4.541787|   -0.365696|    0.249295|
|        2208|  2.2257e-02|   -8.929953|   -4.532034|   -0.365696|    0.249295|
|        2209|  2.2258e-02|   -8.920899|   -4.522981|   -0.365696|    0.249295|
|        2210|  2.2258e-02|   -8.912450|   -4.514532|   -0.365696|    0.249295|
|        2211|  2.2259e-02|   -8.904501|   -4.506584|   -0.365696|    0.249295|
|        2212|  2.2259e-02|   -8.896988|   -4.499071|   -0.365696|    0.249295|
|        2213|  2.2259e-02|   -8.889847|   -4.491931|   -0.365696|    0.249295|
|        2214|  2.2259e-02|   -8.883024|   -4.485108|   -0.365696|    0.249295|
|        2215|  2.2259e-02|   -8.876478|   -4.478561|   -0.365696|    0.249295|
|        2216|  2.2259e-02|   -8.870170|   -4.472254|   -0.365696|    0.249295|
|        2217|  2.2260e-02|   -8.864066|

Now we'll create a convenience function to wrap up all the steps in creating, running, and processing the output of a single simulation. This composed function will then be easy to apply to the sample points in our input - rows in our dataframe.

In [13]:
f = lambda c: cfd.run_simulation(
    save_all(
        geometry.add_mesh(
            make_airplane_model(**c))))
    

In [14]:
df = pandas.DataFrame(
    [dict(chain(
            config.items(),
            {
                "case_name": f"{config['project_name']}-{5*i+j:0>3}",
                "fuselage_radians" : 0.15 + 0.02*i,
                "angle_of_attack" : 0.0 + 3.0 * j,
             }.items())) 
     for i in range(5) for j in range(5)])

### Next run simulations and add columns for extracted output
Now we'll add actual models, config, and simulation results to each row. Here we do it all at once with a the helper function function which composes our geometry, meshing, and simulation steps:

In [ ]:
df = pandas.DataFrame(
    [f(row)
     for row in df.to_dict(orient='records')])

df['aoa_rads'] = df.apply(lambda r:  np.deg2rad(r['angle_of_attack']), axis=1)

Volume: [(3, 1)]
Config: {'meshsize_large': 5.0, 'meshsize_small': 0.8, 'case_index': 0, 'base_dir': '/Users/scot/Projects/Personal/lightningcatcher/notebooks/', 'sim_path': '/Users/scot/Projects/SU2/bin/SU2_CFD', 'mesh_resolution': 0.2, 'mesh_sizemin_scale': 5.0, 'mesh_sizemax_scale': 25.0, 'mesh_distmin_scale': 10.0, 'mesh_distmax_scale': 50.0, 'group_boundary_size': 6.0, 'bounding_box_size': 15.0, 'full_fold_radians': 3.141592653589793, 'wing_tilt_radians': 1.5707963267948966, 'camZoom': 5.0, 'camOffsetX': 0.0, 'camOffsetY': 0.0, 'case_name': 'z-000', 'paper_size_x': 11.0, 'paper_size_z': 4.25, 'paper_thickness': 0.1, 'body_tilt_radians': 0.39269908169872414, 'camX': 30.0, 'camY': 60.0, 'camZ': 0.0, 'camOffsetZ': 0.0, 'project_name': 'z', 'mach_number': 0.01, 'fuselage_radians': 0.15, 'angle_of_attack': 0.0, 'geometry_index': [(3, 1)]}


So we can do the usual things to explore this space. As a two-dimensional response surface, we can plot this as something like a "carpet plot" with our two parameters against Lift (plotted as height) and Drag (plotted as color).

In [ ]:
%matplotlib notebook
fig2 = plt.figure()
ax21 = fig2.add_subplot(projection='3d', azim=20)
ax21.scatter3D(df['angle_of_attack'], df['fuselage_radians'],df['eff_coeff'], 
               s=50, alpha=1.0, c=df['lift_coeff'],
               cmap='plasma')
ax21.set_xlabel('Angle of Attack')
ax21.set_ylabel('Fuselage Radians')
ax21.set_zlabel('Lift/Drag')
plt.show()

! [3d scatterplot of design space samples](simple_3d_scatter.png)

## More visualization

### Flow visualization for each sample point

Add visualization images to each case using an external Visualization tool:

In [ ]:
df['vispath'] = df.apply(lambda row: vis.visualize_case(row['project_name'], row), axis=1)

In [ ]:
# vis.html_imgtable(["two", "twelve", "22"],[df.iloc[2]['vispath'], df.iloc[12]['vispath'], df.iloc[22]['vispath']])
display(HTML(vis.html_imgtable(["two", "twelve", "22"],[df.iloc[2]['vispath'], df.iloc[12]['vispath'], df.iloc[22]['vispath']])))

! [Three flow visualizations](three_flows.png)

### Simple animation across cases

In [ ]:
vis.animate(config, 
                  [df.iloc[2]['vispath'], df.iloc[12]['vispath'], df.iloc[22]['vispath']]
                 )

![GIF animation of deisgn sweep](abc1/abc1.gif)

### Persist Dataframe
We will return to this design space and problem in the "Application" sequel to this article, so it's nice to note that we can store the dataframe for direct manipulation, and it includes everything we'll need if we want to run more simulations later.

In [ ]:
df.to_csv(f"{config['project_name']}/{config['project_name']}_df.csv")
# df = pandas.read_csv(f"{config['project_name']}/{config['project_name']}_df.csv")

# Applications


## Simple Surrogate Models
To wrap up our example of using this dataframe centered approach to exploring a CFD design space, let's make a simple Surrogate model for our simulation and compare the resulting response surface to the actual CFD data.

In [ ]:
%matplotlib notebook
# build the transform to order-2 polynomials and fit a linear model there:
poly = PolynomialFeatures(2)
pt=poly.fit(df[['angle_of_attack', 'fuselage_radians']])
poly_model = LinearRegression()
poly_model.fit(pt.transform(df[['angle_of_attack', 'fuselage_radians']]), df['eff_coeff'])

# Plot the CFD values against the fit model:
fig2 = plt.figure()
ax21 = fig2.add_subplot(projection='3d', azim=20)
ax21.plot_trisurf(df['aoa_rads'], df['fuselage_radians'],
                poly_model.predict(
                    pt.transform(df[['angle_of_attack', 'fuselage_radians']])),alpha=0.90)
ax21.scatter3D(df['aoa_rads'], df['fuselage_radians'],df['eff_coeff'] , 
               s=20, alpha=1.0, 
               c='#FCC')

# Label and Legend
ax21.set_xlabel("Angle of Attack (rads)")
ax21.set_ylabel("Fuselage fold angle")
ax21.set_zlabel("Lift/Drag Ratio")
fig2.show()
fig2.savefig('carpet_plot.png')

! [Carpet plot](carpet_plot.png)

## AeroDB

Precomputed databases of simulation results are useful in many ways. Later we will use one as a lookup table for Realtime Flight Simulation, but for now we simply use one to add more sample points to our our ensemble of simulations. We often want to add more sample points as we look at the results we've generated so far - hopefully we see an interesting area of the design space we'd like to "zoom in" on. We'll add the new sample points as a new dataframe and populate it, then concatenate it with the first and see if we can draw any conclusions from the overall dataset.

In [ ]:

zoomed_params = [geometry.pi/x for x in [15.5, 15.75,16.0,16.25,16.5]]
df2 = pandas.DataFrame(
    [dict(
        chain(
            config.items(),
            {#"project_name": project_name,
                "case_name": f"{config['project_name']}_zoom-{5*i+j:0>3}",
                "fuselage_radians" : zoomed_params[i],
                "angle_of_attack" : 0.0 + 3.0 * j
             }.items()
        )
    ) for i in range(5) for j in range(5)])

df2 = pandas.DataFrame(
    [cfd.single_simulation(
        row, make_airplane_model,
        do_extract=['eff_coeff','lift_coeff', 'drag_coeff', 'momentum_z', 'momentum_x'],
        stdout_to_log=False,
        define_surface_groups=True) 
     for row in df2.to_dict(orient='records')])

df = pandas.concat([df,df2],axis=0)

%matplotlib notebook
fig4 = plt.figure()
ax4 = fig4.add_subplot(projection='3d', azim=20)
ax4.scatter3D(df['angle_of_attack'], df['fuselage_radians'],df['eff_coeff'], 
               s=50, alpha=1.0, c=df['lift_coeff'],
               cmap='plasma')
ax4.set_xlabel('Angle of Attack')
ax4.set_ylabel('Fuselage Radians')
ax4.set_zlabel('Lift/Drag')

At this point, we have a dataframe with all the parameters and simulation outputs as rows. 
Let's add a Angle of Attack in Radians to make it easier to relate to our fold.
Here we pull some interesting values out for a specific value of our Angle of Attack by way of example:

In [ ]:
df[df.angle_of_attack==6][['angle_of_attack', 'aoa_rads', 'fuselage_radians', 'eff_coeff','case_name']]\
    .sort_values(by=['fuselage_radians'])


# Design Optimization

This is all fun, but we haven't answered our question. How should we make that last fold on a "dart" style paper airplane?

What does that even mean. The optimal fold will be corellated with the throw angle (angle of attack) since the angle of the wing means that if thrown parallel to the baseline of the fuselage, the wing will be pointing down and generating negative lift. We calculated for a rangle of throw angles, and don't know what people will actually do, so we'll just average across those and find the best `fuselage_angle` for the resulting one dimensional optimization.

### Simple averaging

We assume a uniform distribution of the release angle (and hence Angle of Attack). This is likely unrealistic, but it means we can just take a simple average across that axis to yield a simple function of the fold angle:

In [ ]:
avg_aoa=df.groupby('fuselage_radians')
# avg_aoa.max()['eff_coeff']
aoa=list(avg_aoa.indices.keys())

Note that if we weight our throw angles to something more realistic we get something fairly close to the experimental values from the above paper, which is nice:

In [ ]:
avg_aoa.apply(lambda x: np.average(x['eff_coeff'],weights=[0.1,0.1,0.4,0.2,0.2]))

In [ ]:
fig3 = plt.figure()
plt.plot(aoa, avg_aoa.max()['eff_coeff'],c='red')
plt.plot(aoa, avg_aoa.min()['eff_coeff'],c='green')
plt.plot(aoa, avg_aoa.apply(lambda x: np.average(x['eff_coeff'],weights=[0.1,0.1,0.4,0.2,0.2])) ,c='blue')
plt.axvline(x=geometry.pi/16.0,ls='--')
fig3.show()
fig3.savefig('eff.png')

This would seem to suggest that folding the wings a little past the fuselage line, rather than the traditional 'dart' fold of $/pi / 16$, is optimal. Not too far though, or the added drag reduces the efficiency. Note however that we're assuming a lot since we don't estimate right on that point. Let's zoom in and add some granularity closer to that critical point:

### Integrate out the fold angle instead - find best launch angle

Taking another literal perspective, we can integrate out the design parameter and take a guess that the optimal angle to throw our paper airplane is around $9^{\circ}$

In [ ]:
fig4=plt.figure()
plt.plot(df.groupby('angle_of_attack').mean()['eff_coeff'])
fig4.show()

# Results

## Do our CFD results seem reasonable?

## Does our optimization seem reasonable?

## How does clever optimization compare to a grid search?

# Open Questions and Next Steps

* Try running the `single_simulation` examples above with `interactive=True`. This will bring up `gmsh` or `paraview` (resp.) to let you move around camera angles and play with parameters, and is a wonderful exploratory mode to work in.
* What is going on with the dip in aerodynamic efficacy as the wing goes just past the exact edge of the fuselage (at $pi/32$). The sharp dip suggests something interesting - very likely, a bug in our geometry code; most likely, the naively simple simulation approach creating unphysical behaviors; or least likely, an actual critical point of some kind in the physics. 

* Should the angle of attack be normalized to the fold angle, so the wing is always at $0^{\circ}$?
* Optimization tools (Dakota, matlab, etc)
* CFD tooling
  * cfd params
  * mesh improvements
  * other CFD platforms
* problem refinement
* more parameter dimensions
* V & V

Now that we can see the general response surface, it would make sense to delve into how to run our simulation enemble more efficiently. We might use "Total Run Time" or something like convergence to generate a response surface while we vary things like mesh size or other alogorithmic parameters. We also can see it would be interesting to explore a larger patch of the `(AoA, FuselageAngle)` space. 

That means we're at a good point to end this notebook - we are exploring properties of the space using dataframes in a notebook in a way that faciliates ensemble thinking. In later notebooks, we'll look at using this ability for a range of applications and modelling improvements.




## simulation optimization

### variable mesh size and granularity

### other algorithmic parameters

### 

## design optimization

What is an interesting AoA to optimize along?

What are useful physical parameters?

Do our result make physical sense?

How should we define fitness?

How should we optimize fitness with our single variable?

Dakota Project

# Colophon


## Workbench - which tools will we use?

We begin using Open Source tools anyone can run on a personal computer and develop a workbench which will allow later exploration of more aggressive (and non-free in all senses) environments and tools.

Readers hailing from the systems side will note that we use shell and file interfaces in some places a cleaner direct API could be used. This is intentional, but not a requirement. It's useful because a number of simulation tools tend to take this configuration file approach and because it makes it's easier to, for example, pull a directory of image files into a movie with `ffmpeg` than to sort out a live interface for doing so. It also makes it easy to decouple components - we try to use light interfaces, so switching CFD engines or visualization should be mostly independent of the high level workflow. It is certainly possible to use clean/direct APIs when available, as we do for interacting with constructive solid geometry for example.

Fundamentally we want to work with Python Notebooks as a way to elegantly tie together multiple tools and platforms into coherent workflows, orchestrate computation, and analyze results. So will prefer tools that best allow this.

## Dataframes and ML
We want to have a familiar way to work with arrays of data and tools for processing them.

A primary goal here is to treat the exploration of a design space as simply mapping problem parameters to objective functions. That is, we'd like to take some geometry and simulation configuration values (input vector) and map them to some estimated physical/computational results (output vector). Note that this is classic Data Science, and so we'd like to keep a DataFrame (Pandas, for now) at the center of our work.

Note that as our design spaces grow, this naturally allows parallelization. Using Spark dataframes, for example, affords scalable parallelism via drop-in replacement and less sophisticated but servicable approaches are possible on cloud platforms.

## Geometry and Meshing
We use `gmsh` for both Parametric CAD and our Meshing. The python interface makes this pleasant-ish We will explore varying geometric parameters by rebuilding geometries and then meshes for different values. We will generate `gmsh` output files as well as....

## CFD

We use SU2 for this first foray.

We plan to make versions with other free options and commercial tools.

## Analysis
We will aggregate the tabular outputs from SU2 and process them using Pandas, Numpy, and other common DS tools . Higher level analysis and visualization will be done with Paraview.

## Optimization

Given a fully populated DOEFrame we can fit a simple model and optimize that using standard tools like `scikit-learn` and similar.